# Heasoftpy testing notebook

This notebook lays out how to verify the functionality in the requirement document here:

https://docs.google.com/document/d/19yX41TdxZRJ037nqk2qJJnyel_SAKqrH1a7koXQ8nU0/edit   


## Import and help

In [1]:
import os, shutil, sys

In [2]:
sys.path.insert(0,"/Users/tjaffe/space/sw/heasoft/")
import heasoftpy as hsp

<font color=green>This correctly shows up the Docstring in the Jupyter Pager for Jupyter Notebook and here inline for JupyterLab.</font>

In [3]:
hsp?

<font color=red>**This works, but it isn't useful to the user.  The Description should include a quick usage guide**, e.g.,
    
    DESCRIPTION

        Python package to wrap the HEASoft tools so that 
        they can be called from python scripts,
        interactive ipython sessions, or Jupyter Notebooks.  

        >>> import heasoftpy as hsp
        >>> help(hsp.fdump)
        ...
        >>> hsp.fhelp(task="fdump")
        >>> hsp.fdump()
        Name of FITS file and [ext#]> Crab_sum_pha.fits
        ...
        >>> result=hsp.fdump(infile='foo.fits',outfile='STDOUT',columns='RATE',rows='1-2',more='yes')
        >>> params2=result.params
        >>> params2['prhead']='no'

        etc., or for scripting with error trapping:
        
          from heasoftpy.core.errors import HeasoftpyExecutionError
          try:
            result=hsp.ftverify(params)
            print("Success")
          except HeasoftpyExecutionError as e:
            print("Error trapped as HeasoftpyExecutionError with message:{}".format(e))
          except Exception as e:
            print("Unexpected status--{}--".format(e))
    
        Version ...


In [4]:
help(hsp)

Help on package heasoftpy:

NAME
    heasoftpy - Python interface to the FTools/HTools package.

DESCRIPTION
    The heasoftpy module provides a Python function corresponding to each
    Heasoft/FTools task. Note that dashes ("-") in task names are replaces with
    underscores ("_") in the corresponding heasoftpy function.
    
    If there is not already a file containing a function to run a given FTools
    program, this will create one upon import. The appropriate parameter file is
    used to do this.
    
    Modification history:
    Version 0.1 ME:    Initial version
    Version 0.1.1 MFC: corrects identification of prompting for required, missing
                       parameters
    Version 0.1.2 ME:  Added Error classes and handling errors when the underlying
                       program encounters an error.
    Version 0.1.3 ME:  Added checks of function versions and replacement of
                       functions that are outdated.
    Version 0.1.4 ME:  Pre-populate the

<font color=red>Should accept a positional argument, i.e., hsp.fhelp("fdump")</font>



In [7]:
##  This doesn't work (2020-08-27).  
#hsp.fhelp("fdump")
print(hsp.fhelp(task="fdump").stdout)

NAME

   fdump -- Convert the contents of a FITS table to ASCII format.

USAGE

        fdump infile[ext#] outfile columns rows

DESCRIPTION

   This task converts the information in the header and data of a FITS
   table extension (either ASCII or binary table) into an ASCII format
   file which then may be displayed, printed, or edited. If desired the
   edited ASCII file may be converted back into a FITS file using the
   fcreate task. By default, fdump will convert all the rows and columns
   of the table extension as well as all the header records to the output
   ASCII file, but only selected information may be output by setting the
   input parameters appropriately. Fdump will fit as many table columns as
   possible on the output page (the page width can be specified with the
   pagewid parameter); any additional columns will be output onto
   subsequent pages of output. The output format is either a default
   format or specified with TDISPn keywords.

   This task is primaril

<font color=green>Prompting now works in a notebook</font>  **<font color=red>But it does not present the default or previously used values to accept, which I think it should the way that other FTOOLS do so you can just hit return.</font>**

<font color=green>These two print the same info on the parameters, though the second shows up in the pager.  </font>

In [13]:
#help(hsp.fdump)
hsp.fdump?

## Running and results

<font color=green>Returned data object:</font>

In [9]:
#  By default, everything to STDOUT
result=hsp.fdump(infile='../tests/my_rate.fit',outfile='STDOUT',columns='ERROR',rows='1-2',more='n',prhead='no')
print(result.stdout)

 
 
        ERROR
        counts/s
      1   1.3094280E+00
      2   1.5160930E+00



In [17]:
#  Prompting
result=hsp.fdump(infile='../tests/my_rate.fit',prhead="no")
print(result.stdout)

Name of optional output file> STDOUT
Names of columns> -
Lists of rows> 1-2
More?> y
 
 
        TIME                    RATE            ERROR
        d                       counts/s        counts/s
      1   7.353720464259241E+02   2.3594639E+01   1.3094280E+00
      2   7.353731574259291E+02   2.3850025E+01   1.5160930E+00



Seeing what's happening with error handling.  This example should produce errors.  

<font color=green>Error successfully trapped as an execution error from the tool</font>

In [18]:
from heasoftpy.core.errors import HeasoftpyExecutionError
try:
    result=hsp.fdump(infile='foo.fits',outfile='STDOUT',columns='-',rows='-',more='y')
    result
except HeasoftpyExecutionError as e:
    print("Error successfully trapped as HeasoftpyExecutionError with message:{}".format(e))
except Exception as e:
    print("Unexpected status--{}--".format(e))

Error successfully trapped as HeasoftpyExecutionError with message:
Error running: fdump
returncode: 1
output:
fdump3.5 : unable to open infile



Ftverify is an interesting case.  It can print things to STDERR but usually returns a good error status anyway.  How is this handled in heasofpy?

Split errors to stderr instead?  <font color=red>Issues with ftverify.</font>

In [19]:
## Tell it to separate out STDERR.  
if os.path.exists("ftverify.out"):
    os.unlink("ftverify.out")
result=hsp.ftverify(
    infile='../tests/my_rate.fit',
    numerrs=10,numwrns=10,
    outfile="ftverify.out")
print(result.stdout)
print("--")
print(result.stderr)

HeasoftpyExecutionError: 
Error running: ftverify
returncode: 6
output:
ERROR: Device not configured
Task ftverify 0.0 terminating with status 6
Unable to redirect prompts to the /dev/tty (at headas_stdio.c:152)


In [20]:
## Tell it to separate out STDERR.  
if os.path.exists("ftverify.out"):
    os.unlink("ftverify.out")
result=hsp.ftverify(
    infile='../tests/my_rate.fit',
    numerrs=10,numwrns=10,
    outfile="ftverify.out",
    stderr=True)
print(result.stdout)
print("--")
print(result.stderr)

HeasoftpyExecutionError: 
Error running: ftverify
returncode: 6
stdout:
Unable to redirect prompts to the /dev/tty (at headas_stdio.c:152)

stderr:
ERROR: Device not configured
Task ftverify 0.0 terminating with status 6


## Parameter interface

### Interactive

<font color=green>This should then prompt for the non-hidden fdump parameters, which it does:</font>
<font color=red>But doesn't work for ftverify</font>

In [22]:
#result=hsp.ftverify()
#print(result.stdout)

Name of FITS file to verify > ../tests/my_rate.fit
number of errors (output) > -
number of warnings (output) > -


HeasoftpyExecutionError: 
Error running: ftverify
returncode: 6
output:
ERROR: Device not configured
Task ftverify 0.0 terminating with status 6
Unable to redirect prompts to the /dev/tty (at headas_stdio.c:152)


<font color=red>**Why does this prompt differently from the command line:**  It looks like heasoftpy is doing the righ tthing and ftverify on the command line is not.  These params have mode a, and the 'mode' parameter itself is set to 'ql'.  So why does ftverify NOT query for them?</font>

<code>
In [4]: hsp.ftverify()
Name of FITS file to verify > foo.fits
number of errors (output) > -
number of warnings (output) > -
Out[4]: Result(returncode=0, stdout=' \n               ftverify 4.18 (CFITSIO V3.420)               \n               ------------------------------               \n \nHEASARC conventions are being checked.\n \nFile: foo.fits\n*** Error:   (from CFITSIO error stack:)\n             failed to find or open the following file: (ffopen)\n             foo.fits\n             \n**** Abort Verification: Fatal Error. ****\n', stderr=None, params={'infile': 'foo.fits', 'numerrs': '-', 'numwrns': '-'}, custom=None)


Suspended
(hspdev) 9:43 ~/space/sw/heasoft > ftverify foo.fits

               ftverify 4.18 (CFITSIO V3.420)
               ------------------------------

HEASARC conventions are being checked.

File: foo.fits
*** Error:   (from CFITSIO error stack:)
             failed to find or open the following file: (ffopen)
             foo.fits

**** Abort Verification: Fatal Error. ****
</code>

<font color=green>It also returns a params dictionary that you can hand it back again</font>

<font color=red> This apparently *used* to work but doesn't anymore.  I guess it used to be a simple dictionary of argument and its value, whereas now it's a dictionary of dictionaries with the full contents of the parameter file.  Hm.</font>

In [30]:
#print(result.params)
#result.params['prhead']='no'
#print(hsp.fdump(**result.params).stdout)

In [36]:
params=result.params
params['prhead']['default']='no'
# Can't do something nice to call it again with that change.

### Learning behavior

<font color=green>This seems to work</font>

In [32]:
## Set up a local pfiles we can mess with
if os.path.exists("pfiles.local"):
    shutil.rmtree("pfiles.local")
os.mkdir("pfiles.local")
os.environ["PFILES"]="./pfiles.local;/Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/syspfiles"
os.listdir("pfiles.local")

[]

In [33]:
os.environ['PFILES']

'./pfiles.local;/Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/syspfiles'

Nothing in the pfiles.  Call with just the required input parameter:

In [34]:
result=hsp.fdump(infile="../tests/my_rate.fit",outfile='STDOUT',columns='-',rows='1-2',more='yes')
result.stdout

"SIMPLE  =                    T / file does conform to FITS standard\nBITPIX  =                    8 / number of bits per data pixel\nNAXIS   =                    0 / number of data axes\nEXTEND  =                    T / FITS dataset may contain extensions\nCOMMENT   FITS (Flexible Image Transport System) format defined in Astronomy and\nCOMMENT   Astrophysics Supplement Series v44/p363, v44/p371, v73/p359, v73/p365.\nCOMMENT   Contact the NASA Science Office of Standards and Technology for the\nCOMMENT   FITS Definition document #100 and other FITS information.\nORIGIN  = 'NASA/GSFC/XTE/MIT_c' / Origin of FITS file\nTELESCOP= 'XTE     '           / Mission Name\nINSTRUME= 'ASM     '           / Instrument Name\nOBJECT  = 'grs1915+105'        / Common Object name\nRA_OBJ  =               288.80 / Object right ascension in degrees\nDEC_OBJ =                10.95 / Object declination in degrees\nEQUINOX =              2000.00 / Equinox for R.A. and Dec.\nRADECSYS= 'FK5     '           / 

Now set a parameter to change its behavior and not print the header keywords.  Make sure that this change is picked up.  Currently, the command line is not constructed with all parameters in the pfile, so this works only because the ftool is doing this behind the scenes.  

In [35]:
os.system("pset fdump prhead=no")

0

<font color=green>The above runs correctly using prhead=no</font>

In [36]:
#  Doesn't work anymore
#  hsp.fdump(**result.params)
result=hsp.fdump(infile="../tests/my_rate.fit",outfile='STDOUT',columns='RATE',rows='1-2',more='yes')
result.stdout

' \n \n        RATE\n        counts/s\n      1   2.3594639E+01\n      2   2.3850025E+01\n'

<font color=green>Check if it learned columns=RATE parameter</yes>

In [37]:
!pget fdump columns

RATE


<font color=red>Now in batch mode, this should *not* know about prhead=no, but it should know about all the other parameters.  This is a mode invented in the requirements doc.  The resulting output file foo.out does not have the header printed, so hsp is taking the prhead parameter from the local PFILES, which is not what I want mode=batch to do.  It should always start from the system pfiles and store no record in the user pfiles.
</font>

In [39]:
result=hsp.fdump(infile="../tests/my_rate.fit",mode="batch",outfile="STDOUT",columns='RATE',rows='1-2',more='yes')
result.stdout

' \n \n        RATE\n        counts/s\n      1   2.3594639E+01\n      2   2.3850025E+01\n'

<font color=red>Do we want basic parameters checked before the system call, e.g., that an integer doesn't get a string or that an infile exists?  This is **not implemented yet** but should give a python exception different from the HeasoftpyExecutionError, such as HeasoftpyParameterError</font>

In [40]:
hsp.fdump(infile="missing_file.fits",pagewidth="foo",outfile='STDOUT',columns='RATE',rows='1-2',more='yes')

HeasoftpyExecutionError: 
Error running: fdump
returncode: 2
output:
fdump3.5 : could not get PAGEWIDTH parameter
fdump3.5 : Error Status Returned :    2
fdump3.5 : unknown error status

fdump3.5 :  ***** FITSIO Error Stack Dump *****
